In [17]:
from dotenv import load_dotenv
load_dotenv()

True

In [2]:

from langchain.document_loaders.generic import GenericLoader
from langchain.document_loaders.parsers import OpenAIWhisperParser
from langchain.document_loaders.blob_loaders.youtube_audio import YoutubeAudioLoader

# `GenericLoader`

In [3]:
# Sal Khan on mastery learning
urls = ["https://youtu.be/lGaQWIV8PZ4"]

# Directory to save audio files
save_dir = "./data/youtube"

# Transcribe the videos to text
loader = GenericLoader(YoutubeAudioLoader(urls, save_dir), OpenAIWhisperParser())
docs = loader.load()

[youtube] Extracting URL: https://youtu.be/lGaQWIV8PZ4
[youtube] lGaQWIV8PZ4: Downloading webpage
[youtube] lGaQWIV8PZ4: Downloading ios player API JSON
[youtube] lGaQWIV8PZ4: Downloading android player API JSON
[youtube] lGaQWIV8PZ4: Downloading m3u8 information
[info] lGaQWIV8PZ4: Downloading 1 format(s): 140
[download] ./data/youtube/Sal Khan's thoughts on mastery learning.m4a has already been downloaded
[download] 100% of    1.86MiB
[ExtractAudio] Not converting audio ./data/youtube/Sal Khan's thoughts on mastery learning.m4a; file is already in target format m4a
Transcribing part 1!
Something False
Response type: <class 'openai.openai_response.OpenAIResponse'>
Transcript {
  "text": "This idea of mastery learning was always kind of this gold standard. This was actually as a part of a fellowship I had while I was at MIT called the El Orante Fellowship to make a learning software for students with ADHD. It is X plus 5 times X plus... And it immediately struck a chord with me because

In [4]:
docs[0]

Document(page_content="This idea of mastery learning was always kind of this gold standard. This was actually as a part of a fellowship I had while I was at MIT called the El Orante Fellowship to make a learning software for students with ADHD. It is X plus 5 times X plus... And it immediately struck a chord with me because going into that, the whole premise of the software that I was working on was this idea that it's not that students aren't capable of learning some advanced mathematics or that the topics are actually difficult, it's more that they just have gaps in their knowledge. And I did a lot of tutoring in high school and I saw that over and over again that the reason why students were having... My friends were having trouble with algebra or geometry, it was just because they had a gap in their negative numbers or dividing decimals or logarithms or whatever else. Good students start failing algebra all of a sudden and start failing calculus all of a sudden, despite it being sm

## Notes
- `YoutubeAudioLoader` downloads every time even if we've got it saved
- `OpenAIWhisperParser` has a chunk size of `20 minutes` because of the `25 MB` limit for the Whisper API

## Wishlist
- [ ] `chunk_duration` in `OpenAIWhisperParser` should be a parameter
- [ ] Get timestamps for words in each chunk or just the chunk

# Replicate x Whisper

## Wishlist
- [ ] When querying these chunks consecutive chunks need to be grouped because the chunks are too small

In [27]:
import replicate
import json

In [ ]:
# output = replicate.run(
#     "openai/whisper:91ee9c0c3df30478510ff8c8a3a545add1ad0259ad3a9f78fba57fbc05ee64f7",
#     input={"audio": open("./data/youtube/Sal Khan's thoughts on mastery learning.m4a", "rb")}
# )
# print(output)
# with open("./data/youtube/Sal Khan's thoughts on mastery learning - Replicate x Whisper API Response.json", 'w') as f:
# 	json.dump(output, f)

In [28]:
output = json.load(open("./data/youtube/Sal Khan's thoughts on mastery learning - Replicate x Whisper API Response.json", 'r'))

In [30]:
list(output.keys())

['detected_language', 'segments', 'transcription', 'translation']

In [31]:
with open("./data/youtube/Sal Khan's thoughts on mastery learning - Transcription.txt", "w") as f:
	f.write(output['transcription'])

In [32]:
output['segments'][0]

{'avg_logprob': -0.3576602739157136,
 'compression_ratio': 1.6016260162601625,
 'end': 7,
 'id': 0,
 'no_speech_prob': 0.00015473616076633334,
 'seek': 0,
 'start': 0,
 'temperature': 0,
 'text': ' This idea of mastery learning was always kind of this gold standard.',
 'tokens': [50364,
  639,
  1558,
  295,
  37951,
  2539,
  390,
  1009,
  733,
  295,
  341,
  3821,
  3832,
  13,
  50714]}

In [38]:
with open("./data/youtube/Sal Khan's thoughts on mastery learning - Segments.txt", "a") as f:
	for chunk in output['segments']:
		f.write(chunk['text'] + "\n")